# HW 6
___

### Hash Function
Write a function **`cu_hash(identikey, modulus)`** that takes a CU Identikey in the form of a string and returns its hash value. Assume that the `identikey` string consists of 4 lower-case letters followed by 4 digits. The hash value is calculated as follows:
* each letter in the Identikey is converted into Unicode by calling `ord()`.
* the four `ord()` values are concatenated to the four digits in the Identikey to create a `hashnum`.
* the function returns `hashnum % modulus` as the hash value.

Example:<br>
`cu_hash('pyth2022', 93)` returns `15` because `hashnum` equals `1121211161042022` and `hashnum % 93` equals `15`.

In [1]:
def cu_hash(identikey, modulus):
    new_str = ''
    for let in identikey[:4]:
        new_str += str(ord(let))
    
    new_str += identikey[4:]
    hashnum = int(new_str)
    
    return hashnum % modulus

In [2]:
cu_hash('pyth2022', 93)

15

### Nodes
Below is a class `Node` that will store a key and data, along with links to neighboring nodes in a `DLinkedList`. 

In [3]:
class Node:
    def __init__(self, key, data):
        self.key = key
        self.data = data
        self.prev = None
        self.next = None

### DLinkedList
Use the `DLinkedList` class (defined in a previous assignment) with the  attribute:
* **`head`** 

and the  methods:
* **`insert`**, **`delete`**, **`search`**, **`keys`**.

In [4]:
class DLinkedList:
    
    def __init__(self):
        self.head = None
        
    def insert(self, node):       # inserting at the head
        node.prev = None
        node.next = self.head
        if self.head != None:
            self.head.prev = node
        self.head = node
        
    def delete(self, node):
        if node.prev == None:
            self.head = node.next
        else:
            node.prev.next = node.next
        if node.next != None:
            node.next.prev = node.prev
    
    def search(self, value):
        x = self.head                    # x is a node
        while x!=None and x.key!=value:
            x = x.next
        return x
    
    def keys(self):
        lst = []
        x = self.head
        while x!=None:
            lst.append(x.key)
            x = x.next
        return lst

### CU Identikey Hash Table
In order to make a hash table for CU student records, create a **`CUHashTable`** class with these attributes:
* **`num_slots`**: number of slots in the table (passed as an argument)
* **`slots`**: a list with `num_slots` elements initialized to `None`. Later these slots will be filled with `DLinkedList`s (defined in a previous assignment).

and these methods:
* **`insert(id, name)`**: takes a student's Identikey and name and does the following:
  * Creates a new `Node` with the `key` and `data` attributes equal to the student Identikey and name, respectively.
  * Calculates the table slot number for the Identikey by calling `cu_hash()` with `num_slots` as the modulus.
  * If the associated table slot is empty, insert a new `DLinkedList` consisting of the new `Node`. 
  * If the associated table slot is already filled, insert the new `Node` to the head of the existing `DLinkedList`.
* **`lookup(id)`**: takes an Identikey and returns the corresponding `Node`. If not found, return `None`.

Example:
```
table = CUHashTable(93)
table.insert('pyth2022', 'Guido Von Rossum')
table.insert('buff8039', 'Ralphie')
table.insert('macu1234', 'Marie Curie')
```
stores the first and third tuples in slot 15, and stores the second tuple in a different slot. Then
```
table.lookup('pyth2022').data
```
returns `'Guido Von Rossum'` and
```
table.lookup('abcd9999')
```
returns `None`.

In [5]:
lst = [None] * 5
lst[3] = 3
lst

[None, None, None, 3, None]

In [6]:
class CUHashTable:
    
    def __init__(self, num_slots):
        self.num_slots = num_slots
        self.slots = [None]*self.num_slots
        
    def insert(self, id, name):
        node = Node(id, name)
        hashnum = cu_hash(id, self.num_slots)
        if self.slots[hashnum] == None:               # if slot is empty
            DLL = DLinkedList()                       # initialize a doubly linked list
            DLL.insert(node)                          # add node to head of DLL
            self.slots[hashnum] = DLL               # make slot equal the DLL
        else:
            self.slots[hashnum].insert(node)         # if slot already has a DLL, insert new node
    
    def lookup(self, id):
        hashnum = cu_hash(id, self.num_slots)
        DLL = self.slots[hashnum]
        if DLL == None:
            return None
        else:
            return DLL.search(id)        # searching the DLL will return the node with 'id' as the key
        

In [7]:
table = CUHashTable(93)
table.insert('pyth2022', 'Guido Von Rossum')
table.insert('buff8039', 'Ralphie')
table.insert('macu1234', 'Marie Curie')

In [8]:
table.lookup('pyth2022').data

'Guido Von Rossum'

In [9]:
table.lookup('abcd9999')

In [10]:
table.lookup('macu1234').data

'Marie Curie'

In [11]:
table.lookup('buff8039').data

'Ralphie'

### Free Ski Pass Giveaway

A free ski pass will be awarded to a lucky CU student. Students who sign up for this free giveaway will be arranged in a line. Then every $k$th student will be eliminated, one at a time, until the lucky winner is left. For example, suppose $k=3$ and there are 10 students named A, B, ..., J. Then eliminating every 3rd student (with wraparound) produces this result with student D as the winner.
```
A B C D E F G H I J
A B   D E   G H   J
A     D E     H   J
      D E         J
      D           J
      D                   
```

Write a function **`giveaway(participants, k)`** that takes a list of participant names, simulates this procedure for a positive integer `k`, and returns the name of the winner. The function should store the names as keys in a `DLinkedList`, then eliminate the participants by removing their corresponding nodes, one at a time.

Example:<br>
`giveaway(list('ABCDEFGHIJ'), 3)` returns `'D'`.

In [101]:
def giveaway(participants, k):
        
    DLL = DLinkedList()
    for name in participants[::-1]:    # populate the DLL with name nodes from participants list
        node = Node(name, None)
        DLL.insert(node)
    
    x = DLL.head
    while DLL.head != None:
        for _ in range(k-1):
            x = x.next if x.next != None else DLL.head
        temp = x.next if x.next != None else DLL.head
        DLL.delete(x)
        x = temp
    
    return x.key

In [102]:
giveaway(list('ABCDEFGHIJ'), 3)

'D'

Read in the file **`giveaway_names.txt`** which contains the names of 100 participants, one name on each line. Store the names in a list called **`participant_names`**. Then run `giveaway(participant_names, 7)`.

In [95]:
with open('giveaway_names.txt') as fp:
    data = fp.readlines()

In [96]:
participant_names = []
for d in data:
    participant_names.append(d.strip())

In [97]:
participant_names[10:17]

['Chelsea', 'Milo', 'Evan', 'Kasey', 'Elizabeth', 'Geraldine', 'Heather']

In [103]:
giveaway(participant_names, 7)

'Lennox'